## Params

In [1]:
seed = 0

class args:
    num_layers = 5
    device = 'cuda:0'
    log_steps = 1
    hidden_channels = 64*4
    dropout = 0.2
    lr = 0.01
    epochs = 1000
    eval_steps = 10
    runs = 1

## Install prerequisites

In [2]:
import time
import humanize
start_time = time.time()

In [3]:
# install Open Graph Benchmark
! pip install ogb

# install PyTorch Geometric
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

     |████████████████████████████████| 45 kB 891 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3 MB 528 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.6 MB 3.3 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.3 MB 4.2 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 6.3 MB 441 kB/s 
You should consider upgra

## Import libraries

In [4]:
import torch
from torch.nn import Parameter
import torch.nn.functional as F

from torch.optim.lr_scheduler import MultiStepLR,CosineAnnealingLR

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

## Logger

In [5]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [6]:
device = f'{args.device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

dataset = PygNodePropPredDataset(name='ogbn-proteins',
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Move edge features to node features.
data.x = data.adj_t.mean(dim=1)
data.adj_t.set_value_(None)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)
data = data.to(device)

Downloaded 0.21 GB: 100%|██████████| 216/216 [00:07<00:00, 28.72it/s]


Extracting dataset/proteinfunc.zip
Processing...
Loading necessary files...
This might take a while.


  0%|          | 0/1 [00:00<?, ?it/s]

Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 382.69it/s]


Converting graphs into PyG objects...
Saving...
Done!


In [7]:
print(humanize.naturaldelta(start_time - time.time()))

4 minutes


## Model Definition

In [8]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(
            GCNConv(in_channels, hidden_channels, normalize=False))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, normalize=False))
        self.convs.append(
            GCNConv(hidden_channels, out_channels, normalize=False))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [9]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [10]:
class WideResGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(WideResGCN, self).__init__()
        self.init = GCNConv(in_channels, in_channels, normalize=True)
        self.left = GCN(in_channels, hidden_channels, out_channels, num_layers, dropout)
        self.right = SAGE(in_channels, hidden_channels, out_channels, num_layers, dropout)
        self.final = GCNConv(out_channels*2, out_channels, normalize=True)
    
    def reset_parameters(self):
        self.init.reset_parameters()
        self.left.reset_parameters()
        self.right.reset_parameters()
        self.final.reset_parameters()
        
    def forward(self, x, adj_t):
        x = self.init(x, adj_t)        
        x_l = self.left(x, adj_t)
        x_r = self.right(x, adj_t)
        x = torch.cat([x_l, x_r],1)
        x = self.final(x, adj_t)
        return x                

## Training and eval

In [11]:
def train(model, data, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = criterion(out, data.y[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [12]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    y_pred = model(data.x, data.adj_t)

    train_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [13]:
# Pre-compute GCN normalization.
adj_t = data.adj_t.set_diag()
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj_t = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
data.adj_t = adj_t
    
evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)
best_test_score = 0


for seed in range(10):
    print("Seed = ",seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    model = GCN(data.num_features, args.hidden_channels, 112, args.num_layers, args.dropout).to(device)
    model.reset_parameters()
    for run in range(args.runs):        
        optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):

            loss = train(model, data, train_idx, optimizer)

            if epoch % args.eval_steps == 0:
                result = test(model, data, split_idx, evaluator)
                logger.add_result(run, result)

                if epoch % args.log_steps == 0:                
                    train_rocauc, valid_rocauc, test_rocauc = result
                    print(f'Run: {run + 1:02d}\t '
                          f'Epoch: {epoch:02d}\t '
                          f'Loss: {loss:.4f}\t '
                          f'Train: {100 * train_rocauc:.2f}%\t '
                          f'Valid: {100 * valid_rocauc:.2f}%\t'
                          f'Test: {100 * test_rocauc:.2f}%')
                    if(test_rocauc > best_test_score):
                        best_test_score = test_rocauc
                        save_path = "gcn.pth"
                        torch.save(model, save_path)
                        print("Model saved.")
        # logger.print_statistics(run)
        torch.cuda.empty_cache()

    # logger.print_statistics()

Seed =  0
Run: 01	 Epoch: 10	 Loss: 0.4590	 Train: 36.74%	 Valid: 35.20%	Test: 38.55%
Model saved.
Run: 01	 Epoch: 20	 Loss: 0.4214	 Train: 37.25%	 Valid: 33.70%	Test: 36.68%
Run: 01	 Epoch: 30	 Loss: 0.4039	 Train: 39.79%	 Valid: 34.34%	Test: 36.73%
Run: 01	 Epoch: 40	 Loss: 0.3932	 Train: 43.54%	 Valid: 34.65%	Test: 36.76%
Run: 01	 Epoch: 50	 Loss: 0.3850	 Train: 49.30%	 Valid: 35.90%	Test: 37.18%
Run: 01	 Epoch: 60	 Loss: 0.3788	 Train: 50.14%	 Valid: 35.87%	Test: 36.79%
Run: 01	 Epoch: 70	 Loss: 0.3717	 Train: 52.46%	 Valid: 37.97%	Test: 37.59%
Run: 01	 Epoch: 80	 Loss: 0.3667	 Train: 53.82%	 Valid: 40.30%	Test: 38.38%
Run: 01	 Epoch: 90	 Loss: 0.3579	 Train: 56.89%	 Valid: 45.64%	Test: 41.10%
Model saved.
Run: 01	 Epoch: 100	 Loss: 0.3541	 Train: 58.07%	 Valid: 49.22%	Test: 42.29%
Model saved.
Run: 01	 Epoch: 110	 Loss: 0.3498	 Train: 58.94%	 Valid: 52.06%	Test: 43.24%
Model saved.
Run: 01	 Epoch: 120	 Loss: 0.3461	 Train: 61.43%	 Valid: 56.03%	Test: 46.07%
Model saved.
Run: 01	 E

In [14]:
print(best_test_score*100)

74.33782905428401


In [15]:
print(humanize.naturaldelta(start_time - time.time()))

3 hours


In [16]:
# del model, optimizer
# torch.cuda.empty_cache()